<a href="https://colab.research.google.com/github/mancai111/Rett_Mouse/blob/main/Other_Original_Design_of_Heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/TestCase/'

import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import dot
from scipy import stats
from numpy.linalg import norm
import matplotlib.pyplot as plt

os.chdir(abspath_curr)
print(os.getcwd())

from Functions import *

/content/drive/My Drive/TestCase


In [ ]:
# NC436 Data

# speed list
#---------------------------------------------------------
# set up parameters

up_speed_list = 'fup60'
down_speed_list = 'fdown60'
speed_list = [up_speed_list, down_speed_list]

basename = 'NC436-M-WT-Day01_aligned.xlsx'

PN_WB = pd.ExcelFile(basename)

# Coordinate Heatmap

In [ ]:
# method

def left_right_paw_heatmap(wb, speed):
    # read the paw movement data
    sample = pd.read_excel(wb, speed+' pawMovement').iloc[:,1:]
    
    # get LX, LY, RX, RY, round them to closet digit, and turn them into numpy form
    LX = sample.iloc[:,0]
    LX = np.rint(LX.to_numpy())
    LY = sample.iloc[:,1]
    LY = np.rint(LY.to_numpy())
    RX = sample.iloc[:,2]
    RX = np.rint(RX.to_numpy())
    RY = sample.iloc[:,3]
    RY = np.rint(RY.to_numpy())
    
    # organize a list of (x,y) coordinates
    coords_list_L = [i for i in zip(LX,LY)]
    coords_list_R = [i for i in zip(RX,RY)]
    coords_list = coords_list_L + coords_list_R

    # count the frequency of each (x,y) coordinate pair
    d = defaultdict(int)
    for i in coords_list:       
        d[i] += 1
        
    coords = [i for i in map(list, zip(*d.keys()))]
    
    # build a dataframe that has x-coord as column and y-coord as row
    column_list = sorted(set(coords[0]))
    index_list = sorted(set(coords[1]))
    df = pd.DataFrame(columns = column_list, index = index_list)

    # fill the values (frequency count)
    key_list, value_list = list(d.keys()), list(d.values())
    for i in range(len(d.keys())):
        df.loc[key_list[i][1], key_list[i][0]] = value_list[i]
    
    for i in np.arange(min(column_list),max(column_list)):
        if i not in df.columns:
            df[i] = np.nan
            
    for i in np.arange(min(index_list),max(column_list)):
        if i not in df.index:
            df.loc[i] = np.nan

    df = df.sort_index(axis=1)
    df = df.sort_index(axis=0)
    df = df.fillna(0)

    # construct heatmap with the dataframe
    fig, ax = plt.subplots(figsize=(15, 10))
    plt.title('Frequncy Count of X-Y Coordinate for Paw Movement at ' + speed)
    sns.heatmap(df)
    plt.ylim(max( max(LY), max(RY) ) - min( min(LY), min(RY) )+1,0)
    plt.show()